In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from sklearn.utils import class_weight
import torch

In [3]:
train_df = pd.read_csv('./dataset/train/train.csv')

In [4]:
with open('./klue-level2-nlp-09/dict_label_to_num.pkl', 'rb') as f:
    label_type = pickle.load(f)
print(label_type)

{'no_relation': 0, 'org:top_members/employees': 1, 'org:members': 2, 'org:product': 3, 'per:title': 4, 'org:alternate_names': 5, 'per:employee_of': 6, 'org:place_of_headquarters': 7, 'per:product': 8, 'org:number_of_employees/members': 9, 'per:children': 10, 'per:place_of_residence': 11, 'per:alternate_names': 12, 'per:other_family': 13, 'per:colleagues': 14, 'per:origin': 15, 'per:siblings': 16, 'per:spouse': 17, 'org:founded': 18, 'org:political/religious_affiliation': 19, 'org:member_of': 20, 'per:parents': 21, 'org:dissolved': 22, 'per:schools_attended': 23, 'per:date_of_death': 24, 'per:date_of_birth': 25, 'per:place_of_birth': 26, 'per:place_of_death': 27, 'org:founded_by': 28, 'per:religion': 29}


In [5]:
train_df['num_label'] = [label_type[t] for t in list(train_df['label'])]

In [6]:
train_df['num_p_label'] = [str(a)+':'+t for a,t in zip(list(train_df['num_label']),list(train_df['label']))]

In [7]:
train_df

,id,sentence,subject_entity,object_entity,label,source,num_label,num_p_label
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia,0,0:no_relation
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree,0,0:no_relation
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,20,20:org:member_of
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,1,1:org:top_members/employees
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia,0,0:no_relation
...,...,...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree,6,6:per:employee_of
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia,14,14:per:colleagues
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree,1,1:org:top_members/employees
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia,0,0:no_relation


In [8]:
class_list = np.array(train_df['num_p_label'])

In [14]:
(train_df.label.value_counts().sort_index(ascending=True))


no_relation                            9534
org:alternate_names                    1320
org:dissolved                            66
org:founded                             450
org:founded_by                          155
org:member_of                          1866
org:members                             420
org:number_of_employees/members          48
org:place_of_headquarters              1195
org:political/religious_affiliation      98
org:product                             380
org:top_members/employees              4284
per:alternate_names                    1001
per:children                            304
per:colleagues                          534
per:date_of_birth                      1130
per:date_of_death                       418
per:employee_of                        3573
per:origin                             1234
per:other_family                        190
per:parents                             520
per:place_of_birth                      166
per:place_of_death              

In [49]:
class_weights=class_weight.compute_class_weight('balanced',np.unique(class_list),class_list)
class_weights=torch.tensor(class_weights,dtype=torch.float)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29], y=[ 0  0 20 ...  1  0  1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [50]:
class_weights

tensor([ 0.1135,  0.2526,  2.5770,  2.8482,  0.5147,  0.8199,  0.3029,  0.9057,
         7.7866, 22.5486,  3.5603,  5.6079,  1.0813,  5.6965,  2.0268,  0.8771,
         7.9583,  1.3614,  2.4052, 11.0442,  0.5800,  2.0814, 16.3990, 13.1992,
         2.5893,  0.9578,  6.5201, 27.0583,  6.9828, 11.2743])

In [8]:
with open('./klue-level2-nlp-09/dict_num_to_label.pkl', 'rb') as f:
    label_type = pickle.load(f)